In [6]:
import sys
import os

# Uzyskaj bieżący katalog roboczy
current_dir = os.getcwd()  # Uzyskaj bieżący katalog roboczy
src_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Ścieżka do katalogu src

# Dodaj katalog src do sys.path
sys.path.append(src_dir)


In [7]:
import logging
logging.basicConfig(level=logging.INFO, filename='flow.log')


In [41]:
from src.flow.PubmedEndpoint import PubmedEndpoint

publication_text = PubmedEndpoint.fetch_full_text_from_pubmed_id('18836447')

In [42]:
with open('../data/pubmed-articles/18836447.txt', 'w') as file:
    file.write(publication_text)


In [10]:
with open('../data/pubmed-articles/23502222.txt', 'r') as file:
    publication_text = file.read()

In [43]:
from src.LlmManager import LlmManager
llm_manager = LlmManager('gpt', 'gpt-4o-mini')

In [12]:
import importlib
import src.flow.coordinates_extraction.CoordinatesRegexExtractor
importlib.reload(src.flow.coordinates_extraction.CoordinatesRegexExtractor) 
from src.flow.coordinates_extraction.CoordinatesRegexExtractor import CoordinatesRegexExtractor

import src.flow.coordinates_extraction.CoordinatesLlmExtractor
importlib.reload(src.flow.coordinates_extraction.CoordinatesLlmExtractor) 
from src.flow.coordinates_extraction.CoordinatesLlmExtractor import CoordinatesLlmExtractor

import src.LlmManager
importlib.reload(src.LlmManager) 
from src.LlmManager import LlmManager



coordinates_regex_extractor = CoordinatesRegexExtractor()
regex_extracted_coordinates = coordinates_regex_extractor.extract_coordinates(publication_text)

llm_manager = LlmManager('gpt', 'gpt-4o-mini')
llm = llm_manager.get_llm()

coordinates_llm_extractor = CoordinatesLlmExtractor(llm)
llm_extracted_coordinates = coordinates_llm_extractor.extract(publication_text)
print("Regex extracted coordinates: ",regex_extracted_coordinates)
print("LLM extracted coordinates: ",llm_extracted_coordinates)
len(llm_extracted_coordinates)

/Users/wojtek/Documents/bioinf/coordinates-lit/src/flow/coordinates_extraction/CoordinatesLlmExtractor.py:41: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=self.llm, prompt=prompt)
/Users/wojtek/Documents/bioinf/coordinates-lit/src/flow/coordinates_extraction/CoordinatesLlmExtractor.py:42: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm_chain.run({


Regex extracted coordinates:  []
LLM extracted coordinates:  []


0

In [13]:
common_extracted_coordinates = set(llm_extracted_coordinates).intersection(set(llm_extracted_coordinates_4o))
print("Wspólne koordynaty wyekstrahowane przez oba modele: ", common_extracted_coordinates)
print(len(common_extracted_coordinates), len(llm_extracted_coordinates), len(llm_extracted_coordinates_4o))

NameError: name 'llm_extracted_coordinates_4o' is not defined

In [14]:
import matplotlib.pyplot as plt
import numpy as np

# Zbierz koordynaty
only_llm_mini = set(llm_extracted_coordinates) - set(llm_extracted_coordinates_4o)
only_llm_4o = set(llm_extracted_coordinates_4o) - set(llm_extracted_coordinates)
common_coordinates = set(llm_extracted_coordinates).intersection(set(llm_extracted_coordinates_4o))
import pandas as pd

# Przygotuj dane do tabeli
data = {
    'Typ': ['Tylko GPT-4o-mini', 'Tylko GPT-4o', 'Wspólne'],
    'Liczba koordynatów': [len(only_llm_mini), len(only_llm_4o), len(common_coordinates)],
    'Koordynaty': [list(only_llm_mini), list(only_llm_4o), list(common_coordinates)]
}

# Tworzenie DataFrame
df = pd.DataFrame(data)

# Wyświetlanie tabeli
print(df)



NameError: name 'llm_extracted_coordinates_4o' is not defined

In [75]:
import json

with open('../data/pubmed-articles/23502222.json', 'r') as file:
    article_data = json.load(file)

# Wyświetlenie załadowanych danych
print(article_data)


[{'pubmed_id': '23502222', 'coordinates': [{'coordinate': 'c.1017+572C>T', 'context': 'recurrent ETS motif point mutation, c.1017+572C>T. The mutation, c.1017+572C>T (i5C>T), is predicted to disrupt an ETS motif within intron 5 following the composite cis-element. Additionally, within the same region, 1 patient was identified with a 28 base deletion of intron 5 that eliminated the E-box and 5 bp of the spacer of the E-box/GATA composite element. 4.II.1\\tT/B/NK/mono\\t26\\tCMV, MAC, recurrent pneumonias, histoplasmosis\\tMDS\\tc.1017+572C>T\\ti5\\t1 25.I.1\\tT/B/NK/mono\\t13\\tHPV, Mycobacterium kansasii, MAC\\tMDS\\tc.1017+572C>T\\ti5\\t 11.II.1\\tB/NK/mono\\t9\\tHPV, VZV, treatment-resistant Candida thrush and vaginitis\\t+Abnormal megakaryocytes\\tc.1017+572C>T\\ti5\\t1 28.I.1\\tB/NK/mono\\t21\\tCMV pneumonia\\t+Abnormal megakaryocytes\\tc.1017+572C>T\\ti5\\t 4.II.5\\tB/NK\\t19\\tHPV\\tMDS\\tc.1017+572C>T\\ti5\\t1 4.I.1\\tMonocytosis\\t78\\tNone\\tCMML\\tc.1017+572C>T\\ti5\\t1 4.III

In [76]:
# Sprawdź, czy wszystkie koordynaty z article_data znajdują się w wyszukiwaniach gpt-4o-mini
article_coordinates = {coord['coordinate'] for article in article_data for coord in article['coordinates']}
gpt_4o_mini_coordinates = set(llm_extracted_coordinates)

# Porównaj koordynaty
missing_coordinates = article_coordinates - gpt_4o_mini_coordinates

if missing_coordinates:
    print("Brakujące koordynaty w wyszukiwaniach gpt-4o-mini:", missing_coordinates)
else:
    print("Wszystkie koordynaty z article_data znajdują się w wyszukiwaniach gpt-4o-mini.")


Brakujące koordynaty w wyszukiwaniach gpt-4o-mini: {'c.761C>T/unknown'}


In [77]:
missing_coordinates_4o = article_coordinates - set(llm_extracted_coordinates_4o)
if missing_coordinates_4o:
    print("Brakujące koordynaty w wyszukiwaniach gpt-4o:", missing_coordinates_4o)
else:
    print("Wszystkie koordynaty z article_data znajdują się w wyszukiwaniach gpt-4o.")



Brakujące koordynaty w wyszukiwaniach gpt-4o: {'c.1-200_871+527 del 2033bp', 'c.761C>T/unknown'}


In [3]:
import json

In [44]:
def analyze_pubmed_coordinates(pubmed_id):
    # Wczytaj dane z pliku JSON
    with open(f'../data/pubmed-articles/{pubmed_id}.json', 'r') as file:
        article_data = json.load(file)

    # Wczytaj tekst publikacji z pliku TXT
    with open(f'../data/pubmed-articles/{pubmed_id}.txt', 'r') as file:
        publication_text = file.read()

    # Wyodrębnij ręcznie koordynaty
    manual_coordinates = {coord['coordinate'] for article in article_data for coord in article['coordinates']}

    # Wyszukaj koordynaty w gpt-4o i gpt-4o-mini
    coordinates_llm_extractor_4o = CoordinatesLlmExtractor(llm_4o)
    llm_extracted_coordinates_4o = coordinates_llm_extractor_4o.extract(publication_text)

    coordinates_llm_extractor_mini = CoordinatesLlmExtractor(llm)
    llm_extracted_coordinates_mini = coordinates_llm_extractor_mini.extract(publication_text)

    print("Liczba wyekstrahowanych koordynatów 4o:", len(llm_extracted_coordinates_4o))
    print("Wyekstrahowane koordynaty 4o:", llm_extracted_coordinates_4o)
    print("Liczba wyekstrahowanych koordynatów mini:", len(llm_extracted_coordinates_mini))
    print("Wyekstrahowane koordynaty mini:", llm_extracted_coordinates_mini)

    # Statystyki
    common_coordinates = manual_coordinates.intersection(set(llm_extracted_coordinates_4o)).intersection(set(llm_extracted_coordinates_mini))
    only_4o_coordinates = set(llm_extracted_coordinates_4o) - set(llm_extracted_coordinates_mini)
    only_mini_coordinates = set(llm_extracted_coordinates_mini) - set(llm_extracted_coordinates_4o)

    print("Wspólne koordynaty:", common_coordinates)
    print("Koordynaty tylko w gpt-4o:", only_4o_coordinates)
    print("Koordynaty tylko w gpt-4o-mini:", only_mini_coordinates)

    # Brakujące koordynaty w porównaniu z ręczną ekstrakcją
    missing_coordinates_4o = manual_coordinates - set(llm_extracted_coordinates_4o)
    missing_coordinates_mini = manual_coordinates - set(llm_extracted_coordinates_mini)


    
    print("Brakujące koordynaty w gpt-4o:", missing_coordinates_4o)
    print("Brakujące koordynaty w gpt-4o-mini:", missing_coordinates_mini)
    return {
       "llm_extracted_coordinates_4o": llm_extracted_coordinates_4o,
       "llm_extracted_coordinates_mini": llm_extracted_coordinates_mini,
       "manual_coordinates": manual_coordinates
    }


In [45]:

def calculate_metrics(tp, tn, fp, fn):
    # Obliczanie metryk
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall
    }




In [66]:
result = analyze_pubmed_coordinates('21441262')

Liczba wyekstrahowanych koordynatów 4o: 10
Wyekstrahowane koordynaty 4o: ['c.700T>C', '14q12-q13.1', '2q11.2', 'c.263_278del16', 'c.460dupG', 'c.256C>T', 'c.757A>G', 'c.505_506delGGinsT', 'chr14:28,754,647-28,756,495 bp', '14q12']
Liczba wyekstrahowanych koordynatów mini: 7
Wyekstrahowane koordynaty mini: ['PRKD1', '14q12-q13.1', 'C14orf23', '2;14', 'FOXG1', '2q11.2', '14q12']
Wspólne koordynaty: {'14q12-q13.1', '14q12'}
Koordynaty tylko w gpt-4o: {'c.700T>C', 'chr14:28,754,647-28,756,495 bp', 'c.263_278del16', 'c.256C>T', 'c.505_506delGGinsT', 'c.757A>G', 'c.460dupG'}
Koordynaty tylko w gpt-4o-mini: {'PRKD1', 'FOXG1', 'C14orf23', '2;14'}
Brakujące koordynaty w gpt-4o: {'transitions c.757A>G (p.N253D)', '~5.4-Mb heterozygous deletion in 14q12-q13.1', '46,XX,t(2;14)(q13;q13)', '~1.09-Mb microdeletion downstream of FOXG1', '~10 kb gene-poor region in 14q12 located ~265 kb downstream of FOXG1', 'c.256C>T (p.R86X)', 'c.263_278del16 (p.R88PfsX99)', '1-Mb region downstream of FOXG1', 'c.460d

In [69]:
process_result(result)

4o:  ['c.700T>C', '14q12-q13.1', '2q11.2', 'c.263_278del16', 'c.460dupG', 'c.256C>T', 'c.757A>G', 'c.505_506delGGinsT', 'chr14:28,754,647-28,756,495 bp', '14q12']
mini:  ['PRKD1', '14q12-q13.1', 'C14orf23', '2;14', 'FOXG1', '2q11.2', '14q12']
manual:  {'transitions c.757A>G (p.N253D)', '46,XX,t(2;14)(q13;q13)', '~5.4-Mb heterozygous deletion in 14q12-q13.1', '~1.09-Mb microdeletion downstream of FOXG1', '~10 kb gene-poor region in 14q12 located ~265 kb downstream of FOXG1', 'c.256C>T (p.R86X)', 'c.263_278del16 (p.R88PfsX99)', '1-Mb region downstream of FOXG1', '14q12-q13.1', 'c.460dupG mutation', '28,930,280-28,932,099 bp', 'c.700T>C (p.S234P)', 'chr14:28,930,280-28,932,099', 'c.460dupG (p.E154GfsX301)', '46,XX,del(14)(q13.1q13.2)', 'chr14:28,754,647-28,756,495', '14q12'}
Koordynaty istniejące w gpt-4o, ale nieistniejące w ręcznej ekstrakcji: {'c.700T>C', 'chr14:28,754,647-28,756,495 bp', '2q11.2', 'c.263_278del16', 'c.256C>T', 'c.505_506delGGinsT', 'c.757A>G', 'c.460dupG'}
TP: {'14q12

0.14814814814814817

In [68]:
def process_result(result):
    print("4o: ", result['llm_extracted_coordinates_4o'])
    print("mini: ", result['llm_extracted_coordinates_mini'])
    print("manual: ", result['manual_coordinates'])
    
    only_llm_4o_not_in_manual = set(result['llm_extracted_coordinates_4o']) - set(result['manual_coordinates']) 
    print("Koordynaty istniejące w gpt-4o, ale nieistniejące w ręcznej ekstrakcji:", only_llm_4o_not_in_manual)
    
    llm_extracted_coordinates_4o = result['llm_extracted_coordinates_4o']
    manual_coordinates = result['manual_coordinates']
    
    tp = set(llm_extracted_coordinates_4o) & set(manual_coordinates)  # TP: koordynaty, które są w obu zbiorach
    tn = 0  # TN: koordynaty manualne, które nie są w 4o
    fp = set(llm_extracted_coordinates_4o) - set(manual_coordinates)  # FP: koordynaty 4o, które nie są w manualnych
    fn = len(set(manual_coordinates) - set(llm_extracted_coordinates_4o))

    print("TP:", tp)
    print("TN:", tn)
    print("FP:", fp)
    print("FN:", fn)

    print(calculate_metrics(len(tp), tn, len(fp), fn))
    return calculate_metrics(len(tp), tn, len(fp), fn)['F1 Score']


# Example usage
# process_result(result)






In [55]:
def process_result_mini(result):
    print("mini: ", result['llm_extracted_coordinates_mini'])
    print("manual: ", result['manual_coordinates'])
    
    only_llm_mini_not_in_manual = set(result['llm_extracted_coordinates_mini']) - set(result['manual_coordinates']) 
    print("Koordynaty istniejące w gpt-mini, ale nieistniejące w ręcznej ekstrakcji:", only_llm_mini_not_in_manual)
    
    llm_extracted_coordinates_mini = result['llm_extracted_coordinates_mini']
    manual_coordinates = result['manual_coordinates']
    
    tp = len(set(llm_extracted_coordinates_mini) & set(manual_coordinates))  # TP: koordynaty, które są w obu zbiorach
    tn = 0  # TN: koordynaty manualne, które nie są w mini
    fp = len(set(llm_extracted_coordinates_mini) - set(manual_coordinates))  # FP: koordynaty mini, które nie są w manualnych
    fn = len(set(manual_coordinates) - set(llm_extracted_coordinates_mini))

    print("TP:", tp)
    print("TN:", tn)
    print("FP:", fp)
    print("FN:", fn)

    print(calculate_metrics(tp, tn, fp, fn))
    return calculate_metrics(tp, tn, fp, fn)['F1 Score']

In [61]:
f1s

[0, 0, 0, 0]

In [15]:
llm_manager_4o = LlmManager('gpt', 'gpt-4o')
llm_4o = llm_manager_4o.get_llm()

coordinates_llm_extractor_4o = CoordinatesLlmExtractor(llm_4o)
llm_extracted_coordinates_4o = coordinates_llm_extractor_4o.extract(publication_text)
print("LLM 4o extracted coordinates: ",llm_extracted_coordinates_4o)
len(llm_extracted_coordinates_4o)


LLM 4o extracted coordinates:  []


0

In [62]:
import os

# Ścieżka do folderu z artykułami
folder_path = '../data/pubmed-articles'

# Wyciągnij wszystkie pubmed ids z nazw plików
pubmed_ids = [filename.split('.')[0] for filename in os.listdir(folder_path) if filename.endswith('.txt')]

print("Wyciągnięte pubmed ids:", pubmed_ids)


Wyciągnięte pubmed ids: ['18836447', '22996659', '21441262', '23502222']


In [65]:
f1s = []
results = []
f1s_mini = []
for pubmed_id in pubmed_ids:
    result = analyze_pubmed_coordinates(pubmed_id)
    results.append(result)
    f1 = process_result(result)
    f1_mini = process_result_mini(result)
    f1s.append(f1)
    f1s_mini.append(f1_mini)

print("F1 scores:", f1s)


Liczba wyekstrahowanych koordynatów 4o: 1
Wyekstrahowane koordynaty 4o: ['460kb upstream of SHH']
Liczba wyekstrahowanych koordynatów mini: 2
Wyekstrahowane koordynaty mini: ['6.5 kb downstream of SBE2', '10 bp-block of SBE2 sequence']
Wspólne koordynaty: set()
Koordynaty tylko w gpt-4o: {'460kb upstream of SHH'}
Koordynaty tylko w gpt-4o-mini: {'6.5 kb downstream of SBE2', '10 bp-block of SBE2 sequence'}
Brakujące koordynaty w gpt-4o: set()
Brakujące koordynaty w gpt-4o-mini: set()
4o:  ['460kb upstream of SHH']
mini:  ['6.5 kb downstream of SBE2', '10 bp-block of SBE2 sequence']
manual:  set()
Koordynaty istniejące w gpt-4o, ale nieistniejące w ręcznej ekstrakcji: {'460kb upstream of SHH'}
TP: 0
TN: 0
FP: 1
FN: 0
{'F1 Score': 0, 'Precision': 0.0, 'Recall': 0}
mini:  ['6.5 kb downstream of SBE2', '10 bp-block of SBE2 sequence']
manual:  set()
Koordynaty istniejące w gpt-mini, ale nieistniejące w ręcznej ekstrakcji: {'6.5 kb downstream of SBE2', '10 bp-block of SBE2 sequence'}
TP: 0
TN

## Jak przeprowadzić analizę benchmarkową?
1. Ściągnij pliki tekstowe dla podanych pubmed ids
2. Wygeneruj koordynaty dla każdego z plików
3. 

In [66]:
import pandas as pd
pd.DataFrame(llm_extracted_coordinates_4o)

,0
0,c.1187G>A
1,c.1192C>T
2,c.243_244delAinsGC
3,c.1163T>C
4,c.1061C>T
5,c.586_593dup
6,c.302delG
7,c.761C>T
8,c.1017+572C>T
9,c.769_778dup


In [67]:
import json
with open('../data/pubmed-articles/21441262.json', 'r') as file:
    y = json.load(file)

In [68]:
y_coordinates = list(map(lambda x: x['coordinate'], next(article for article in y if article['pubmed_id'] == '21441262')['coordinates']))
y_coordinates


['46,XX,t(2;14)(q13;q13)',
 '14q12-q13.1',
 '14q12',
 'chr14:28,754,647-28,756,495 bp',
 '28,930,280-28,932,099 bp',
 '~10 kb gene-poor region in 14q12 located ~265 kb downstream of FOXG1',
 '1-Mb region downstream of FOXG1',
 'chr14:28,930,280-28,932,099',
 '46,XX,del(14)(q13.1q13.2)',
 '~5.4-Mb heterozygous deletion in 14q12-q13.1',
 '~1.09-Mb microdeletion downstream of FOXG1',
 'transitions c.757A>G (p.N253D)',
 'c.700T>C (p.S234P)',
 'c.460dupG (p.E154GfsX301)',
 'c.256C>T (p.R86X)',
 'c.263_278del16 (p.R88PfsX99)',
 'c.460dupG mutation']

In [69]:
from src.ContextExtractionService import ContextExtractionService

context_extraction_service = ContextExtractionService('gpt-4o-mini')
coordinate = coordinates_list[3]
coordinate_context = context_extraction_service.extract_context_from_coordinate(publication_text, coordinate)
print(coordinate_context)

ModuleNotFoundError: No module named 'src.ContextExtractionService'

In [5]:
import importlib
import src.SequenceOntologyMappingService  # Initial import
importlib.reload(src.SequenceOntologyMappingService)  # Reload the module
from src.SequenceOntologyMappingService import SequenceOntologyMappingService  # Re-import the class

sequence_ontology_mapping_service = SequenceOntologyMappingService('gpt-4o-mini')
sequence_ontology_term = sequence_ontology_mapping_service.map_coordinates_to_sequence_ontology(coordinate, coordinate_context)
print(sequence_ontology_term)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


```json
{"name": "minus_1_frameshift_variant", "definition": "\"A sequence variant which causes a disruption of the translational reading frame, by shifting one base ahead.\" [http://arjournals.annualreviews.org/doi/pdf/10.1146/annurev.ge.08.120174.001535]", "comment": null, "_distance": 156.83042907714844}
```


In [17]:
import src.LinksFromQueryExtractionService
importlib.reload(src.LinksFromQueryExtractionService)
from src.LinksFromQueryExtractionService import LinksFromQueryExtractionService

links_from_query_extraction_service = LinksFromQueryExtractionService('gpt-4o-mini')
user_query_dict = {'type of variant': 'frameshift'}
links = links_from_query_extraction_service.extract_links_from_query(coordinate, coordinate_context, sequence_ontology_term, user_query_dict)

Link between type of variant and frameshift: False. Type of relation is RELATION_TYPE  . Reasoning: The context indicates that the mutations listed (including chr10:23508365A>G and chr10:23508446A>C) do not specifically refer to a frameshift variant. Instead, the provided sequence ontology describes a "minus_1_frameshift_variant," which is a specific type of sequence variant that disrupts the translational reading frame. There is no direct evidence in the context provided that links the user value "frameshift" specifically to the mentioned coordinates as a confirmed type of variant.


In [10]:
coordinate_context

'The following sentences from the publication are relevant to the genomic coordinates chr10:23508365A>G and chr10:23508446A>C:\n\n1. "We also identified a homozygous mutation in one patient with pancreatic agenesis and intrahepatic cholestatic failure."\n2. "Of the 10 probands with mutations in this element, 6 had the same chr10:23508437A>G mutation as part of a shared extended haplotype (minimal shared haplotype of 1.2Mb)."\n3. "Three of the remaining probands had different base substitution mutations: a homozygous chr10:23508363A>G mutation, a homozygous chr10:23508305A>G mutation and compound heterozygous chr10:23508365A>G/chr10:23508446A>C mutations."\n4. "Testing of parents and siblings demonstrated co-segregation of the mutations with diabetes and exocrine insufficiency."\n\nThese sentences provide context about the specific mutations at the given coordinates, their occurrence in patients with pancreatic agenesis, and the analysis of genetic inheritance related to these mutations